## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-05-19-51-37 +0000,nyt,CBS Abruptly Changes Editing Rules After Attac...,https://www.nytimes.com/2025/09/05/business/cb...
1,2025-09-05-19-44-35 +0000,nyt,U.S. Is Increasingly Exposed to Chinese Electi...,https://www.nytimes.com/2025/09/05/us/politics...
2,2025-09-05-19-42-04 +0000,bbc,Google fined €2.95bn by EU for abusing adverti...,https://www.bbc.com/news/articles/c1wgn3lre14o...
3,2025-09-05-19-42-03 +0000,nypost,Trump slashes tariffs on Japanese cars after s...,https://nypost.com/2025/09/05/business/trump-c...
4,2025-09-05-19-40-00 +0000,wsj,U.S. job growth continued to slow down in Augu...,https://www.wsj.com/economy/jobs/jobs-report-a...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,59
49,new,13
169,up,13
822,department,12
190,will,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
289,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...,123
283,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...,122
268,2025-09-05-02-44-00 +0000,wsj,President Trump was within his authority when ...,https://www.wsj.com/politics/national-security...,107
277,2025-09-05-01-45-00 +0000,wsj,President Trump on Thursday led leaders of the...,https://www.wsj.com/politics/trump-tech-ceo-ro...,102
190,2025-09-05-10-29-41 +0000,nyt,Trump to Sign Order Renaming the Defense Depar...,https://www.nytimes.com/2025/09/04/us/politics...,102


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
289,123,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...
116,67,2025-09-05-15-41-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
81,60,2025-09-05-17-04-18 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
283,57,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...
238,48,2025-09-05-07-53-33 +0000,wsj,Stock Market Today: Weak Jobs Data Boosts Rate...,https://www.wsj.com/livecoverage/jobs-report-a...
72,46,2025-09-05-17-29-00 +0000,wsj,Putin Says Russia Will Strike Any Western Troo...,https://www.wsj.com/world/russia/putin-says-ru...
321,45,2025-09-04-22-12-54 +0000,nypost,"Biden White House lawyers, DOJ were befuddled ...",https://nypost.com/2025/09/04/us-news/biden-wh...
341,44,2025-09-04-21-25-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr. was gri...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
21,41,2025-09-05-19-11-36 +0000,nyt,Israel Steps Up Attacks on Gaza City Ahead of ...,https://www.nytimes.com/2025/09/05/world/middl...
118,40,2025-09-05-15-39-10 +0000,nyt,"Angela Rayner, UK Deputy Prime Minister, Resig...",https://www.nytimes.com/2025/09/05/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
